<a href="https://colab.research.google.com/github/gomesluiz/pln-na-pratica/blob/main/03-pre-processamento-de-dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Técnicas de Pré-processamento

In [ ]:
# Bibliotecas que precisam ser instaladas para a prática:
!pip install nltk==3.8.1
!pip install spacy==3.7.2
!pip install unidecode==1.3.8
!pip install wikipedia==1.4.0
!pip install matplotlib==3.7.1

In [ ]:
# Declara define funções utilitárias utilizadas no notebook.
import datetime
import sys

import matplotlib.pyplot as plt
from nltk.probability import FreqDist

def formata_msg(nivel, msg):
    """
    Formata uma mensagem de log incluindo o nível de severidade, timestamp
    e a mensagem.

    Parâmetros:
    - nivel (str): Nível de severidade da mensagem (ex: 'INFO', 'ERROR', 'WARNING').
    - msg (str): A mensagem de log propriamente dita.

    Retorna:
    - str: A mensagem de log formatada.
    """
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    return f"[{nivel}] {timestamp} - {msg}"


def calcula_pct_reducao(valor_inicial, valor_final):
  """
  Calcula a porcentagem de redução de um valor inicial para um valor final.

  Args:
      valor_inicial (float): O valor inicial antes da redução.
      valor_final (float): O valor após a redução.

  Returns:
      float: A porcentagem de redução do valor inicial para o valor final.
  """
  return (valor_inicial - valor_final)/valor_inicial * 100


def plot_frequencia_tokens(tokens, ax, title):
    """
    Plota a frequência dos 20 tokens mais comuns em um gráfico de barras horizontais,
    formatado de acordo com os princípios de storytelling.

    Gera um gráfico de barras horizontais no objeto de eixo do Matplotlib fornecido,
    destacando os tokens mais comuns e suas frequências, com foco em clareza e
    comunicação eficaz dos dados.

    Parâmetros:
    - tokens (list): Lista de tokens para análise de frequência.
    - ax (matplotlib.axes.Axes): Eixo onde o gráfico será plotado.
    - title (str, opcional): Título do gráfico.

    Retorna:
    - None

    Exemplo:
    >>> fig, ax = plt.subplots()
    >>> plot_frequencia_tokens(['a', 'b', 'a'], ax)
    """
    fd_words = FreqDist(tokens)
    most_common = fd_words.most_common(20)
    words, frequencies = zip(*most_common)
    y_pos = range(len(words))

    ax.barh(y_pos, frequencies, align='center', color='skyblue')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(words, fontsize=10)
    ax.invert_yaxis()  # Inverte o eixo y para que os valores mais altos fiquem no topo

    for i, v in enumerate(frequencies):
        ax.text(v + 0.1, i, str(v), color='black', va='center', fontsize=8)

    ax.set_xlabel('Frequência', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(axis='x', linestyle='--', alpha=0.6)  # Adiciona uma grade sutil no eixo x

    # Remove as bordas superior e direita para um visual mais limpo
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

print(formata_msg("INFO", "Funções utilitárias prontas para utilização."))
print(formata_msg("INFO", f"Versão do Python: {sys.version} "))

In [ ]:
# Importa módulos essenciais para funcionalidades do notebook.
import re
import string

import nltk
from nltk.corpus import stopwords

import spacy
from unidecode import unidecode
import wikipedia

print(formata_msg("INFO", "Pacotes importadas com sucesso; ambiente pronto."))

In [ ]:
# Baixa as bibliotecas necessárias do NLTK e do Spacy
nltk.download("stopwords")        # Baixa lista de stopwords
nltk.download('punkt')            # Baixa dados para o tokenizador
nltk.download('rslp')             # Baixa o stemmer RSLP para o português
!spacy download pt_core_news_sm   # Baixa o modelo de linguagem pré-treinado fornecido pela biblioteca spaCy

print(formata_msg("INFO", "Download das bases do NLTK e Spacy realizado com sucesso."))

## Wikipedia

A Wikipedia é uma enciclopédia digital gratuita e de código aberto que permite aos usuários ler e editar seu conteúdo. Representando a maior e mais popular fonte de referência na internet, destaca-se pelo seu modelo colaborativo, no qual voluntários de todo o mundo contribuem com informações em vários idiomas. Para acessar e interagir com o conteúdo da Wikipedia usando Python, pode-se utilizar o pacote "wikipedia", uma biblioteca Python desenvolvida para simplificar a busca e recuperação de artigos, além de facilitar a obtenção de resumos, links e outras informações disponíveis na Wikipedia.



In [ ]:
# Define o idioma da Wikipedia para Português
wikipedia.set_lang("pt")

# Busca a página da Wikipedia sobre Processamento de Linguagem Natural (PLN)
pln = wikipedia.page("PLN")

# Extrai o conteúdo da página como um corpus de texto
corpus = pln.content

print(formata_msg("INFO",f"1000 caracteres do conteúdo textual da URL {pln.url}:\n"))
print("--- INÍCIO ---")
print(pln.content[0:1000])
print("--- TÉRMINO ---")

## 1. Tokenização
A tokenização é o processo inicial de pré-processamento de textos, que consiste na divisão do conteúdo em unidades menores, como palavras, termos ou tokens. Essas unidades podem incluir palavras, números e sinais de pontuação, essenciais para a compreensão e análise de textos em linguagem natural. A identificação precisa desses elementos é crucial para o processamento eficaz do texto. Existem diversas técnicas para realizar a tokenização, incluindo:

* Utilização da função split() para separar o texto em tokens baseando-se em espaços ou outros delimitadores.
* Uso de expressões regulares (Regex) para uma tokenização mais detalhada e flexível.
* Emprego da biblioteca Natural Language Toolkit (NLTK), especializada em processamento de linguagem natural, que fornece ferramentas avançadas para tokenização, entre outras funções de processamento de texto.


### 1.1. Método **string.split()** da biblioteca padrão

In [ ]:
# Divide o corpus em tokens e armazena na lista
tokens_split = corpus.split()

# Exibe os tokens e o número total de tokens
print(formata_msg("INFO", f"Tokens com o string.split(): \n{tokens_split}\n"))
print(formata_msg("INFO", f"Tamanho dos tokens com o string.split(): {len(tokens_split)}"))

### 1.2. Método **re.findall()** da biblioteca padrão


In [ ]:
# Padrão regular para capturar tokens:
    # \w+          - Sequência de caracteres alfanuméricos/underscore
    # (?:'\w+)?    - Opcional: apóstrofo seguido de caracteres alfanuméricos
    # |            - OU
    # [^\w\s]      - Um único caractere que não é alfanumérico ou espaço em branco
padrao = r"\w+(?:'\w+)?|[^\w\s]"

# Utiliza a expressão regular para encontrar todos os tokens no corpus
tokens_regex = re.findall(padrao, corpus)

print(formata_msg("INFO", f"Tokens com o re.findall(): \n{tokens_regex}\n"))
print(formata_msg("INFO", f"Tamanho: {len(tokens_regex)}"))

### 1.3. Método **nltk.word_tokenize()**


In [ ]:
# Tokeniza o corpus utilizando a função word_tokenize do NLTK
# Configura o idioma para português
tokens_nltk = nltk.word_tokenize(corpus, language='portuguese')

# Exibe os tokens extraídos e a quantidade total
print(formata_msg("INFO", f"Tokens com nltk.word_tokenize(): \n{tokens_nltk}\n"))
print(formata_msg("INFO", f"Tamanho dos tokens nltk.word_tokenize(): {len(tokens_nltk)}"))

### 1.4. Gráfico de frequências de tokens por método de tokenização



In [ ]:
# Criando uma figura e três subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # Ajuste o tamanho conforme necessário

# Plotando as frequências para cada lista de tokens em um subplot diferente
plot_frequencia_tokens(tokens_split, axs[0], f"Top 20 tokens mais comuns\nsplit")
plot_frequencia_tokens(tokens_regex, axs[1], f"Top 20 tokens mais comuns\nregex")
plot_frequencia_tokens(tokens_nltk, axs[2], f"Top 20 tokens mais comuns\nnltk")

plt.tight_layout()  # Ajusta automaticamente os parâmetros do subplot
plt.show()  # Mostra a figura com os três gráficos lado a lado

Como observado, há variações nas tokenizações realizadas, sendo a que utilizou expressões regulares (regex) a que gerou mais tokens. Portanto, vamos utilizar essa abordagem a partir de agora, com os resultados obtidos por **tokens_regex**.

## 2. Capitalização com **str.lower()**


In [ ]:
# Utiliza uma compreensão de lista para converter cada token para minúsculo
tokens_minusculos = [token.lower() for token in tokens_regex]

# Exibe os tokens convertidos
print(formata_msg("INFO", f"Tokens em minúsculos: \n{tokens_minusculos}\n"))

## 3. Remoção stopwords com a NLTK


In [ ]:
# Carrega a lista de stopwords para o idioma especificado.
portugues_stops = stopwords.words('portuguese')

# Filtra os tokens, removendo aqueles que são stopwords.
tokens_sem_stop = [token for token in tokens_regex_minusculos if token not in portugues_stops]

# Exibe os tokens extraídos e a quantidade total
print(formata_msg("INFO", f"Tokens sem stopwords: \n{tokens_sem_stop}\n"))
print(formata_msg("INFO", f"Tamanho dos tokens sem stopwords: {len(tokens_sem_stop)}"))
print(formata_msg("INFO", f"Redução de tokens em relação à quantidade anterior em: {calcula_pct_reducao(len(tokens_regex_minusculos), len(tokens_sem_stop)):.2f}%"))

## 4. Remoção números com regex

In [ ]:
# Filtra os tokens, removendo aqueles que são números.
tokens_sem_numeros = [re.sub(r'\d+', '', palavra) for palavra in tokens_sem_stop if re.sub(r'\d+', '', palavra)]

# Exibe os tokens extraídos e a quantidade total
print(formata_msg("INFO", f"Tokens sem números: \n{tokens_sem_numeros}\n"))
print(formata_msg("INFO", f"Tamanho dos tokens sem números: {len(tokens_sem_numeros)}"))
print(formata_msg("INFO", f"Redução de tokens em relação à quantidade anterior em: {calcula_pct_reducao(len(tokens_sem_stop), len(tokens_sem_numeros)):.2f}%"))

## 5. Remoção pontuação com **string.punctuation**


In [ ]:
# Filtra os tokens, removendo aqueles que são pontuações.
tokens_sem_pontuacao = [token for  token in tokens_sem_numeros if token not in string.punctuation]

print(formata_msg("INFO", f"Tokens sem pontuações: \n{tokens_sem_pontuacao}\n"))
print(formata_msg("INFO", f"Tamanho dos tokens sem pontuacoes: {len(tokens_sem_pontuacao)}"))
print(formata_msg("INFO", f"Redução de tokens em relação à quantidade anterior em: {calcula_pct_reducao(len(tokens_sem_numeros), len(tokens_sem_pontuacao)):.2f}%"))

## 6. Remoção acentos com o **unidecode(str)**


In [ ]:
tokens_sem_acentos = [unidecode(token) for token in tokens_sem_pontuacao]

print(formata_msg("INFO", f"Tokens sem acentos: \n{tokens_sem_acentos}\n"))
print(formata_msg("INFO", f"Tamanho dos tokens sem acentos: {len(tokens_sem_pontuacao)}"))
print(formata_msg("INFO", f"Redução de tokens em relação à quantidade anterior em: {calcula_pct_reducao(len(tokens_sem_pontuacao), len(tokens_sem_acentos)):.2f}%"))

### 6.1. Gráfico de frequência dos tokens sem acentos.

In [ ]:
# Criando uma figura e três subplots
fig, axs = plt.subplots(1, 1, figsize=(10, 6))  # Ajuste o tamanho conforme necessário

# Plotando as frequências para cada lista de tokens em um subplot diferente
plot_frequencia_tokens(tokens_sem_acentos, axs, f"Top 20 tokens mais comuns\napós a remoção dos acentos")

plt.tight_layout()  # Ajusta automaticamente os parâmetros do subplot
plt.show()  # Mostra a figura com os três gráficos lado a lado

## 8. Stemming com a NLTK

In [ ]:
# Inicialize o stemmer RSLPStemmer do NLTK
stemmed = nltk.stem.RSLPStemmer()

# Aplique o stemmer a cada token da lista, gerando uma nova lista de tokens 'stemmed'
tokens_stemmed = [stemmed.stem(token) for token in tokens_sem_acentos]

print(formata_msg("INFO", f"Tokens sem acentos: \n{tokens_stemmed}\n"))

### 8.1. Plote a frequência dos tokens após o processo de stemming.

In [ ]:
# Criando uma figura e três subplots
fig, axs = plt.subplots(1, 1, figsize=(10, 6))  # Ajuste o tamanho conforme necessário

# Plotando as frequências para cada lista de tokens em um subplot diferente
plot_frequencia_tokens(tokens_stemmed, axs, f"Top 20 tokens mais comuns\napós a remoção dos acentos")

plt.tight_layout()  # Ajusta automaticamente os parâmetros do subplot
plt.show()  # Mostra a figura com os três gráficos lado a lado

## 9. Lemmatization com o Spacy


In [ ]:
# Carrega o modelo de lematização para português
nlp = spacy.load("pt_core_news_sm")

In [ ]:
# Junte os tokens em uma única string, separada por espaços.
str_tokens = ' '.join(tokens_sem_pontuacao)

# Processa a string de tokens com o modelo de linguagem natural.
doc = nlp(str_tokens)

# Gera os lemas dos tokens processados.
tokens_lemm = [token.lemma_ for token in doc]


print(formata_msg("INFO", f"Tokens lematizados: \n{tokens_lemm}\n"))

### 9.1. Plote a frequência dos tokens após o processo de stemming..

In [ ]:
# Criando uma figura e três subplots
fig, axs = plt.subplots(1, 1, figsize=(10, 6))  # Ajuste o tamanho conforme necessário

# Plotando as frequências para cada lista de tokens em um subplot diferente
plot_frequencia_tokens(tokens_lemm, axs, f"Top 20 tokens mais comuns\napós a lematização")

plt.tight_layout()  # Ajusta automaticamente os parâmetros do subplot
plt.show()  # Mostra a figura com os três gráficos lado a lado